# Implementación de la metahuristica Busqueda Armonica Global GBHS

In [1]:
# Librerias y dependencias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
from sklearn.metrics import f1_score, accuracy_score
import sys
import math

In [2]:
sys.float_info.max

1.7976931348623157e+308

In [3]:
# Dataframes resultantes de la Fase 1
# Grupos de Calidad con su respctivo modelo y valor de correlación

GC1 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo1.xlsx",index_col=0)
GC2 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo2.xlsx",index_col=0)
GC3 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo3.xlsx",index_col=0)

In [4]:
print(f"Grupo de Calidad 1 {len(GC1)}, Grupo de Calidad 2: {len(GC2)}, Grupo de Calidad 3: {len(GC3)}")

Grupo de Calidad 1 398, Grupo de Calidad 2: 84, Grupo de Calidad 3: 317


In [5]:
# Se define la etiqueta de c/ grupo
GC1["Grupo"] = 1
GC2["Grupo"] = 2
GC3["Grupo"] = 3


In [6]:
# Se Unen los diferentes Grupos en un unico dataset
df = pd.concat([GC1, GC2,GC3], ignore_index=True)

In [7]:
df.shape

(799, 177)

In [8]:
# Se eliminan algunas variables no relevantes
df = df.drop(["ID_LOTE", "RDT_AJUSTADO"], axis=1)

In [9]:
df.head(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
0,6,45,77,54000,11,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,5,54,80,60000,10,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
2,4,48,82,55000,12,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,1
3,5,47,73,70000,72,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
4,5,47,81,67000,14,1,0,0,0,0,...,0,0,0,0,1,1,0,1,0,1


In [10]:
df.tail(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
794,6,50,75,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
795,5,48,88,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
796,6,49,87,60000,17,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,3
797,7,47,84,70000,18,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
798,6,49,79,61000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3


In [11]:
df.values[:]

array([[ 6., 45., 77., ...,  1.,  0.,  1.],
       [ 5., 54., 80., ...,  1.,  0.,  1.],
       [ 4., 48., 82., ...,  1.,  0.,  1.],
       ...,
       [ 6., 49., 87., ...,  1.,  0.,  3.],
       [ 7., 47., 84., ...,  1.,  0.,  3.],
       [ 6., 49., 79., ...,  1.,  0.,  3.]])

In [12]:
df.values[:,:-1]

array([[ 6., 45., 77., ...,  0.,  1.,  0.],
       [ 5., 54., 80., ...,  0.,  1.,  0.],
       [ 4., 48., 82., ...,  0.,  1.,  0.],
       ...,
       [ 6., 49., 87., ...,  0.,  1.,  0.],
       [ 7., 47., 84., ...,  0.,  1.,  0.],
       [ 6., 49., 79., ...,  0.,  1.,  0.]])

In [13]:
# Se realiza la normalización del dataset    [Todos los datos, exepto ultima columa de la clase del grupo]
norm = MinMaxScaler()
df_norm = norm.fit_transform(df.values[:,:-1])

In [14]:
df_norm

array([[0.4       , 0.3220339 , 0.46268657, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.47457627, 0.50746269, ..., 0.        , 1.        ,
        0.        ],
       [0.26666667, 0.37288136, 0.53731343, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.4       , 0.38983051, 0.6119403 , ..., 0.        , 1.        ,
        0.        ],
       [0.46666667, 0.3559322 , 0.56716418, ..., 0.        , 1.        ,
        0.        ],
       [0.4       , 0.38983051, 0.49253731, ..., 0.        , 1.        ,
        0.        ]])

In [15]:
print(len(df_norm))

799


In [46]:

def generarVectorPesos(w):
    #w = np.random.rand(dim)
    posceros = np.random.choice(len(w), 50, replace=False)
    w[posceros] = 0
    print(w)
    s  = np.sum(w)
    # Se agregan 2 decimales
    wf = np.round(w/s, 3)
    sc = 1- np.sum(wf)
    pos = np.random.randint(len(wf))
    if sc != 0:
        wf[pos]= wf[pos]+sc

    return wf


In [47]:
wp = np.random.rand(df_norm.shape[1])
generarVectorPesos(wp)

[0.60253573 0.31722508 0.51571183 0.31217125 0.         0.20994846
 0.12801046 0.92154273 0.         0.7341072  0.03910657 0.18449512
 0.8032224  0.         0.58835825 0.17646898 0.0799226  0.91399084
 0.2259421  0.         0.97246596 0.28934117 0.         0.
 0.88404541 0.81699586 0.75648257 0.29572728 0.25113933 0.75126509
 0.06188203 0.96397537 0.15291661 0.03832899 0.         0.
 0.6378374  0.88870531 0.6942224  0.30950574 0.         0.
 0.         0.83096467 0.60736152 0.57707508 0.44754738 0.46842906
 0.94198172 0.81977275 0.09638492 0.         0.31517698 0.
 0.45176475 0.22756417 0.37660991 0.         0.96993976 0.21900551
 0.72878692 0.         0.         0.         0.99142421 0.97192939
 0.         0.         0.62889733 0.86293317 0.88073302 0.5191846
 0.99729676 0.         0.         0.         0.27061526 0.
 0.72281697 0.22941348 0.93674505 0.         0.         0.73478946
 0.72573526 0.77076246 0.73943014 0.91057261 0.31322035 0.97250691
 0.         0.02901723 0.42730272 0.

array([0.01 , 0.005, 0.008, 0.005, 0.   , 0.003, 0.002, 0.015, 0.   ,
       0.012, 0.001, 0.003, 0.013, 0.   , 0.009, 0.003, 0.001, 0.015,
       0.004, 0.   , 0.015, 0.005, 0.   , 0.   , 0.014, 0.013, 0.012,
       0.005, 0.004, 0.012, 0.001, 0.015, 0.002, 0.001, 0.   , 0.   ,
       0.01 , 0.014, 0.011, 0.005, 0.   , 0.   , 0.   , 0.013, 0.01 ,
       0.009, 0.007, 0.007, 0.015, 0.013, 0.002, 0.   , 0.005, 0.   ,
       0.007, 0.004, 0.006, 0.   , 0.015, 0.003, 0.012, 0.   , 0.   ,
       0.   , 0.016, 0.016, 0.   , 0.   , 0.01 , 0.014, 0.014, 0.008,
       0.016, 0.   , 0.   , 0.   , 0.004, 0.   , 0.012, 0.004, 0.015,
       0.   , 0.   , 0.012, 0.012, 0.012, 0.012, 0.015, 0.005, 0.015,
       0.   , 0.   , 0.007, 0.002, 0.012, 0.002, 0.012, 0.007, 0.007,
       0.   , 0.01 , 0.   , 0.012, 0.008, 0.015, 0.007, 0.003, 0.007,
       0.   , 0.005, 0.01 , 0.015, 0.   , 0.005, 0.   , 0.004, 0.005,
       0.   , 0.   , 0.003, 0.006, 0.007, 0.004, 0.006, 0.   , 0.007,
       0.006, 0.015,

### Construcción de la Función de Calidad (Qs)
**Parametros de entrada**
- df_norm: Dataset normalizado, sin tener en cunta la clase
- wi: Vector de pesos generado aleatroiamente

**Parametros de salida**
- Calidad (Qs) asociada a ese vector de pesos (wi)



In [28]:
# Dependeindo del vector de pesos me extrae el acuracy - F1 score
def qualityFunction(df_norm, wi):

    # Vector Distancias Euclideanas Ponderadas
    y_pred = []
    minDep = sys.float_info.max
    posMinDep = 0
    for i in range(len(df_norm)):
        vrf = df_norm[i] 
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sqrt(np.sum(ri))
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
              
        y_pred.append(df.values[posMinDep][-1])
    qs = accuracy_score(df.values[:,-1], y_pred)
    return qs

In [23]:
np.power([2,3, 4], 2)

array([ 4,  9, 16], dtype=int32)

### Construcción de la memoria Armonica
**Parametros de entrada**
- MAC: Numero de armonisas que se van almacenar en la  memoria [5,10,15,20] 

**Parametros de salida**
-  MA : Memoria armonica ordenada deacuerdo al fitnes (Qs) de mayor a menor

In [25]:

def GenerateArmonyMemory(MAC):
    Lw = []
    for i in range (MAC):
        # 1. Generar pesos aleatorios | dim = cantidad de caracteristicas
        vp = np.random.rand(df_norm.shape[1])
        wi = generarVectorPesos(vp)
        # 2. Se obtiene el fitnes asociados a esos pesos.
        Qs = qualityFunction(df_norm, wi)
        wiq = np.append(wi, Qs)
        Lw.append(wiq)

    
    # Se obtiene la memoria armonica ordenada de  Mayor- Menor
    #MA = Lw.sort(key=lambda x: x[-1], reverse=True)
    return Lw

In [48]:
# Genero la memoria Armonica
MA = GenerateArmonyMemory(5)


[9.89722444e-01 2.05705848e-01 2.10467464e-01 0.00000000e+00
 8.74671893e-01 2.25365517e-01 4.75527731e-01 6.20688384e-01
 5.39884631e-01 9.43963784e-01 5.07371119e-01 9.62348151e-01
 8.85584501e-01 0.00000000e+00 1.52156898e-01 8.80068451e-01
 8.68764921e-01 9.86844595e-01 0.00000000e+00 4.01803264e-01
 4.10067110e-01 9.63446365e-01 4.70648566e-01 9.72451786e-01
 9.02878200e-02 0.00000000e+00 0.00000000e+00 2.90536564e-01
 0.00000000e+00 2.28487416e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 8.91704246e-02 7.92675197e-01 0.00000000e+00
 3.85921327e-01 4.72114962e-03 6.95662073e-01 1.22370388e-01
 0.00000000e+00 0.00000000e+00 1.89881660e-01 6.25697424e-01
 0.00000000e+00 0.00000000e+00 3.50847049e-01 8.82537102e-01
 4.84872478e-02 0.00000000e+00 1.25530208e-01 8.50613570e-01
 5.30127726e-01 7.30501724e-01 0.00000000e+00 6.80403805e-01
 1.66700234e-01 9.81075118e-01 7.55140157e-01 5.19479126e-01
 2.23802396e-01 4.98343056e-02 7.65413703e-01 4.10328736e-01
 0.00000000e+00 5.722091

In [49]:
for j in range(len(MA)):
    print(MA[j][0], end=' ')
    print(MA[j][1], end=' ')
    print(MA[j][2], end=' ')
    print("Fitnes: ",MA[j][-1])


0.015 0.003 0.003 Fitnes:  0.17772215269086358
0.002 0.008 0.002 Fitnes:  0.17772215269086358
0.012 0.0 0.007 Fitnes:  0.5794743429286608
0.0 0.015 0.0 Fitnes:  0.5794743429286608
0.014 0.0 0.0 Fitnes:  0.43178973717146435


In [50]:
# Ordeno la memoria Armonica
MA.sort(key=lambda x: x[-1], reverse=True)
MA

[array([0.012     , 0.        , 0.007     , 0.007     , 0.013     ,
        0.007     , 0.        , 0.015     , 0.007     , 0.        ,
        0.01      , 0.        , 0.011     , 0.006     , 0.016     ,
        0.001     , 0.003     , 0.        , 0.008     , 0.001     ,
        0.        , 0.009     , 0.008     , 0.015     , 0.        ,
        0.002     , 0.009     , 0.        , 0.011     , 0.004     ,
        0.014     , 0.        , 0.012     , 0.001     , 0.        ,
        0.006     , 0.002     , 0.002     , 0.003     , 0.009     ,
        0.002     , 0.01      , 0.014     , 0.005     , 0.012     ,
        0.016     , 0.003     , 0.001     , 0.008     , 0.004     ,
        0.015     , 0.006     , 0.014     , 0.015     , 0.009     ,
        0.006     , 0.015     , 0.014     , 0.01      , 0.015     ,
        0.005     , 0.015     , 0.        , 0.        , 0.        ,
        0.006     , 0.        , 0.005     , 0.        , 0.        ,
        0.014     , 0.        , 0.006     , 0.  

### Implementacion GBHS
**Parametros de entrada**
- Imp: Numero de improvisaciones (iteraciones) que realiza GBHS.
- ParMin: Tasa de ajuste de tono mínima.
- ParMax: Tasa de ajuste de tono máxima.
- HCMR: Tasa de consideración de memoria armónica.
- p: Numero de atributos dataset Original o Normalizado



In [51]:
lmp = 10
P= len(MA[0])
HMRC = 0.85
PAR = 0.35              #[0.1, 0.12, 0.13 , 0.40]
# Tamaño de la memoria armonica
hmns = 5

In [35]:
MA[4][174]

0.17772215269086358

In [52]:
import random

curva = []
for i in range (lmp):
    print(f"Iteracion {i}")
    pesosAleatorios = np.random.rand(P-1)
    # print("Vector de Pesos: ", pesosAleatorios)
    # Donde P es el numero de variables
    for j in range(P-1):
        Aleatorio1 = random.random() 
        if (Aleatorio1 < HMRC):
            # pma: Numero entrero entre 0 y (hmns -1)
            # hmns: Tamaño de la memoria armonica
            pma = random.randint(0, hmns-1)
            pesosAleatorios[j]= MA[pma][j]

            Aleatorio2 = random.random()
            if Aleatorio2 < PAR:
                # Se toma el valor de la mejor armonia
                pesosAleatorios[j] = MA[0][j]
        
        else:

            Aleatorio3 = random.random()/(P/2)
            pesosAleatorios[j] = Aleatorio3
    
    # Fin primer for - Conformación vector de pesosS
    wf = generarVectorPesos(pesosAleatorios)
    fitnes = qualityFunction(df_norm, wf)
    print("Funcion de calidad: ", fitnes)


    # Remplazo

    if MA[hmns -1][P-1] < fitnes:
        # Creo el nuevo registro y  remplazo por el peor
        new_register = np.append(wf, fitnes)
        # Remplazo
        MA[hmns-1] = new_register
        # Ordeno la Armonia
        print("------------------------------------")
        MA.sort(key=lambda x: x[-1], reverse=True)
    
    else:
        print("La funcion de calidad del ultimo armonico  es mejor que el fitnes")
    
    curva.append(MA[0][P-1])
    


    


Iteracion 0
[0.00000000e+00 0.00000000e+00 0.00000000e+00 2.69182811e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.00000000e-03 1.32258942e-03 2.24795047e-05
 2.47258716e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.00000000e-03 0.00000000e+00
 0.00000000e+00 1.00000000e-03 0.00000000e+00 0.00000000e+00
 5.68029390e-03 6.60023743e-03 9.00000000e-03 3.00000000e-03
 1.10000000e-02 4.00000000e-03 1.00000000e-02 5.00000000e-03
 1.20000000e-02 0.00000000e+00 0.00000000e+00 6.00000000e-03
 0.00000000e+00 2.00000000e-03 8.61260799e-03 9.00000000e-03
 1.60000000e-02 0.00000000e+00 3.00000000e-03 1.10000000e-02
 0.00000000e+00 1.60000000e-02 3.23536380e-04 1.00000000e-03
 9.38527354e-03 0.00000000e+00 0.00000000e+00 6.00000000e-03
 4.36576094e-03 0.00000000e+00 6.02382081e-03 0.00000000e+00
 0.00000000e+00 1.40000000e-02 0.00000000e+00 8.00000000e-03
 1.40000000e-02 0.00000000e+00 1.06976304e-02 6.00000000e-03
 4.67540256e

C:\Users\germanm\AppData\Local\Temp\ipykernel_20056\219039592.py:13: RuntimeWarning: invalid value encountered in sqrt
  dE = np.sqrt(np.sum(ri))


Funcion de calidad:  0.574468085106383
La funcion de calidad del ultimo armonico  es mejor que el fitnes
Iteracion 8
[0.         0.         0.007      0.018      0.         0.007
 0.004      0.02       0.         0.         0.         0.
 0.         0.00911398 0.         0.001      0.00672367 0.
 0.017      0.         0.         0.         0.         0.028
 0.         0.015      0.         0.00091828 0.         0.
 0.014      0.         0.         0.         0.         0.
 0.         0.008      0.007      0.         0.041      0.009
 0.021      0.007      0.         0.         0.004      0.004
 0.007      0.         0.         0.008      0.006      0.015
 0.         0.         0.         0.011      0.         0.
 0.         0.01022137 0.         0.         0.         0.
 0.         0.013      0.         0.         0.         0.
 0.         0.         0.         0.001      0.005      0.
 0.         0.         0.         0.         0.014      0.
 0.         0.012      0.004      0.004   

In [53]:
curva

[0.5794743429286608,
 0.5794743429286608,
 0.5807259073842302,
 0.8560700876095119,
 0.8560700876095119,
 0.8560700876095119,
 0.8560700876095119,
 0.8560700876095119,
 0.8560700876095119,
 0.8610763454317898]

In [39]:
MA

[array([ 0.001     ,  0.007     ,  0.001     ,  0.009     ,  0.01      ,
         0.005     ,  0.006     ,  0.002     ,  0.001     ,  0.002     ,
         0.002     ,  0.01      ,  0.006     ,  0.009     ,  0.002     ,
         0.01      ,  0.009     ,  0.01      ,  0.009     ,  0.007     ,
         0.        ,  0.002     ,  0.009     ,  0.006     ,  0.004     ,
         0.035     ,  0.        ,  0.        ,  0.009     ,  0.006     ,
         0.008     ,  0.009     ,  0.002     ,  0.004     ,  0.001     ,
         0.01      ,  0.007     ,  0.005     ,  0.        ,  0.004     ,
         0.005     ,  0.003     ,  0.009     ,  0.003     ,  0.        ,
         0.012     ,  0.007     ,  0.007     ,  0.007     ,  0.007     ,
         0.007     ,  0.004     ,  0.        ,  0.        ,  0.006     ,
         0.009     ,  0.009     ,  0.004     ,  0.003     ,  0.002     ,
         0.005     ,  0.002     ,  0.004     ,  0.007     ,  0.01      ,
         0.        ,  0.009     ,  0.001     ,  0.0

In [ ]:
curva

In [75]:
c = np.array([1.3,4])

In [77]:
d = np.append(c, 5)

In [78]:
d

array([1.3, 4. , 5. ])

In [71]:
MA[0][174]

0.5031289111389237